## Retrieving pdfs from Gallica
The code is a simplified version from the one made for [pyllica](https://api.bnf.fr/fr/extracteur-python-de-corpus-de-periodiques) and posted [here](https://colab.research.google.com/drive/19qt44UwQgu4l0cMME4afEhQQh2swNwmy?usp=sharing) by Dorialexander

In [67]:
import urllib.request, urllib.error, urllib.parse
import pandas as pd
import re
import os
import time

Mount Drive for direct downloads

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
%cd /content/drive/MyDrive/work/PaPa/gallica_files

/content/drive/MyDrive/work/PaPa/gallica_files


In [43]:
%ls

metadata.csv


Read metadata with links & filenames

In [54]:
metadata = pd.read_table('metadata.csv', sep=';')
metadata.head()

,id,link_bnf,link_gallica,author_short,selected_for_galerie,title,date,format,author,contributors,editors,description,subject,notes
0,1,http://catalogue.bnf.fr/ark:/12148/cb37236775k,https://gallica.bnf.fr/ark:/12148/bpt6k42803h,Bancal des Issarts,yes,"Opinion sur le divorce, prononcée au Conseil d...",1797,57p___1 microfiche ; 105*148 mm,"Bancal Desissarts, Jean-Henri (1750-1826). Aut...",NaN,Micro Graphix,Collection : Les archives de la Révolution fra...,Divorce -- Droit -- France,NaN
1,2,http://catalogue.bnf.fr/ark:/12148/cb346933758,https://gallica.bnf.fr/ark:/12148/bpt6k85298x,Bancal des Issarts,yes,Du nouvel ordre social ([Reprod.]) par Henry B...,1792,"52p___1 microfiche acétate de 98 images, diazo...","Bancal Desissarts, Jean-Henri (1750-1826). Aut...",NaN,Hachette (Paris),NaN,NaN,NaN
2,3,http://catalogue.bnf.fr/ark:/12148/cb30051725x,https://gallica.bnf.fr/ark:/12148/bpt6k42324856,Bancal des Issarts,NaN,Convention nationale. Discours et projet de dé...,1792,"In-8° , 28 p.","Bancal Desissarts, Jean-Henri (1750-1826). Aut...",France. Convention nationale. Éditeur scientif...,Impr. nationale (Paris),NaN,NaN,NaN
3,4,http://catalogue.bnf.fr/ark:/12148/cb300517148,https://gallica.bnf.fr/ark:/12148/bpt6k853041,Bancal des Issarts,NaN,"Henri Bancal, député à la Convention, à Anacha...",1793-1794,16 p. ; in-8,"Bancal Desissarts, Jean-Henri (1750-1826). Aut...",NaN,Impr. du Cercle social (Paris),NaN,NaN,NaN
4,5,http://catalogue.bnf.fr/ark:/12148/cb30051721j,https://gallica.bnf.fr/ark:/12148/bpt6k62669573,Bancal des Issarts,NaN,"Des prochaines élections de l'an VI , par Jean...",1798,12 p. ; in-8,"Bancal Desissarts, Jean-Henri (1750-1826). Aut...",NaN,impr. de Baudouin (Paris),NaN,NaN,NaN


In [78]:
metadata['id'] = metadata.id.astype(str)

# create a column for filenames
metadata['filename'] = metadata.apply(lambda x: '_'.join(x[['author_short', 'id']]), axis = 1)

# clean columns with multiple Gallica links (several exemplaires digitised for one catalogue entry)
metadata['link_gallica'] = metadata['link_gallica'].str.split(' | ').str[0]

In [61]:
# extract filenames as a list
fh = metadata['filename'].to_list()
print(fh[1:5])

['Bancal des Issarts_2', 'Bancal des Issarts_3', 'Bancal des Issarts_4', 'Bancal des Issarts_5']


In [80]:
# extract links as a list
links = metadata['link_gallica'].to_list()
print(links[100:150])

['https://gallica.bnf.fr/ark:/12148/bpt6k474686', 'https://gallica.bnf.fr/ark:/12148/bpt6k47048g', 'https://gallica.bnf.fr/ark:/12148/bpt6k40875w', 'https://gallica.bnf.fr/ark:/12148/bpt6k47953j', 'https://gallica.bnf.fr/ark:/12148/bpt6k4103z', 'https://gallica.bnf.fr/ark:/12148/bpt6k5735739b', 'https://gallica.bnf.fr/ark:/12148/bpt6k5736010r', 'https://gallica.bnf.fr/ark:/12148/bpt6k58125365', 'https://gallica.bnf.fr/ark:/12148/bpt6k133810j', 'https://gallica.bnf.fr/ark:/12148/bpt6k5626256x', 'https://gallica.bnf.fr/ark:/12148/bpt6k1095385b', 'https://gallica.bnf.fr/ark:/12148/bpt6k1506575p', 'https://gallica.bnf.fr/ark:/12148/bpt6k6558248s', 'https://gallica.bnf.fr/ark:/12148/bpt6k56833m', 'https://gallica.bnf.fr/ark:/12148/bpt6k568072', 'https://gallica.bnf.fr/ark:/12148/bpt6k56854k', 'https://gallica.bnf.fr/ark:/12148/bpt6k567560', 'https://gallica.bnf.fr/ark:/12148/bpt6k432691', 'https://gallica.bnf.fr/ark:/12148/bpt6k43219g', 'https://gallica.bnf.fr/ark:/12148/bpt6k43259q', 'http

In [72]:
# test print out
for i in range(5):
  url = links[i]
  pdf_file = fh[i]+'.pdf'
  final_url = url+'.pdf'
  print(url)
  print(final_url)
  print(pdf_file)
  print(time.ctime())
  time.sleep(5)
  print("waited 5 sec, time now: ", time.ctime())

https://gallica.bnf.fr/ark:/12148/bpt6k42803h
https://gallica.bnf.fr/ark:/12148/bpt6k42803h.pdf
Bancal des Issarts_1.pdf
Fri Nov 10 17:26:16 2023
waited 5 sec, time now:  Fri Nov 10 17:26:21 2023
https://gallica.bnf.fr/ark:/12148/bpt6k85298x
https://gallica.bnf.fr/ark:/12148/bpt6k85298x.pdf
Bancal des Issarts_2.pdf
Fri Nov 10 17:26:21 2023
waited 5 sec, time now:  Fri Nov 10 17:26:26 2023
https://gallica.bnf.fr/ark:/12148/bpt6k42324856
https://gallica.bnf.fr/ark:/12148/bpt6k42324856.pdf
Bancal des Issarts_3.pdf
Fri Nov 10 17:26:26 2023
waited 5 sec, time now:  Fri Nov 10 17:26:31 2023
https://gallica.bnf.fr/ark:/12148/bpt6k853041
https://gallica.bnf.fr/ark:/12148/bpt6k853041.pdf
Bancal des Issarts_4.pdf
Fri Nov 10 17:26:31 2023
waited 5 sec, time now:  Fri Nov 10 17:26:36 2023
https://gallica.bnf.fr/ark:/12148/bpt6k62669573
https://gallica.bnf.fr/ark:/12148/bpt6k62669573.pdf
Bancal des Issarts_5.pdf
Fri Nov 10 17:26:36 2023
waited 5 sec, time now:  Fri Nov 10 17:26:41 2023


In [77]:
# test download
error_list = []

for i in range(16, 20):
  url = links[i]
  pdf_file = fh[i]+'.pdf'
  final_url = url+'.pdf'
  try:
    response = urllib.request.urlopen(url)
    urllib.request.urlretrieve(final_url, pdf_file)
  except:
    print("Something went wrong with URL:", url, "\nError for the filename: ", pdf_file, "\n")
    error_list = error_list.append(url)
  print("Finished with:", pdf_file, "\nTime:", time.ctime(), "\n")
  time.sleep(5)

Finished with: Baudeau_23.pdf 
Time: Fri Nov 10 17:42:34 2023 

Finished with: Baudeau_25.pdf 
Time: Fri Nov 10 17:44:31 2023 

Finished with: Baudeau_26.pdf 
Time: Fri Nov 10 17:44:49 2023 

Finished with: Baudeau_28.pdf 
Time: Fri Nov 10 17:45:14 2023 

